In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from gensim.models import Word2Vec as wv

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
#import PhysicallyInformedLossFunction as PhysLoss



In [ ]:
#Opening contents of Word2Vec model1
data = '/Users/Thomas/Desktop/BETO2020-master/Ant_Syn_Scraping/all_abstracts_model'
os.chdir(data)
model1 = wv.load('all_abstract_model.model')
vocabulary1 = list(model1.wv.vocab)
#use model.build_vocab(sentence, update=True) to add missing words to model's vocabulary?
#or delete the rows that yield the KeyError?

In [ ]:
#Opening contents of Word2Vec model2
data = '/Users/Thomas/Desktop/BETO2020-master/Ant_Syn_Scraping/3j_15e_50D_min1_model'
os.chdir(data)
model2 = wv.load('3j_15e_50D_min1_model.model')
vocabulary2 = list(model2.wv.vocab)

In [ ]:
data = '/Users/Thomas/Desktop/BETO2020-master/Ant_Syn_Scraping/'
os.chdir(data)
carbon_df = pd.read_excel('Carbon_SynAntList_Full_Refined_copy.xlsx', skip_rows=1, nrows=2000, index_col=0)
carbon_df = carbon_df.rename(columns = {'Carbon_SynAntList_Full_Refined':'index', 'Unnamed: 1':'word 1', 'Unnamed: 2':'word 2','Unnamed: 3':'relationship', 'Unnamed: 4': 'label'})
carbon_df = carbon_df.fillna(0)
carbon_df = carbon_df[1:]

#finding which words are in the pd but not in vocabulary1
list1 = list(carbon_df['word 1'])
list2 = list(carbon_df['word 2'])
missing = list((set(list1).difference(vocabulary1))) + list((set(list2).difference(vocabulary1)))

#keeping only the rows in the pd that have words in vocabulary1
carbon_df = carbon_df[~carbon_df['word 1'].isin(missing)]
carbon_df = carbon_df[~carbon_df['word 2'].isin(missing)]

#reseting indeces after mask
carbon_df.reset_index(inplace = True)


In [ ]:
for i in range(len(carbon_df)):
    carbon_df['word 1'].iloc[i] = model1.wv.__getitem__(str(carbon_df['word 1'].iloc[i]))
    carbon_df['word 2'].iloc[i] = model1.wv.__getitem__(str(carbon_df['word 2'].iloc[i]))
    
    if carbon_df['relationship'].iloc[i] == 'syn' and carbon_df['label'].iloc[i] == 1:
        carbon_df['relationship'].iloc[i] = 1
    else:
        carbon_df['relationship'].iloc[i] = 0      
    

In [ ]:
carbon_df.to_json('SYN_NN_DATA.json')

In [2]:
carbon_df = pd.read_json('SYN_NN_DATA.json', dtype = np.float32)

In [3]:
#Hyper parameters
num_epochs = 100
batch_size = 50
learning_rate = 0.008

# Device configuration (GPU if available, otherwise CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
#unzipping the vectors for 'word 1' and 'word 2' into different dataframes

x1_train_array = np.empty((len(x_train),50))
x1_train_df = pd.DataFrame(data=x1_train_array[0:,:])

x1_test_array = np.empty((len(x_test),50))
x1_test_df = pd.DataFrame(data=x1_test_array[0:,:])

x2_train_array = np.empty((len(x_train),50))
x2_train_df = pd.DataFrame(data=x2_train_array[0:,:])

x2_test_array = np.empty((len(x_test),50))
x2_test_df = pd.DataFrame(data=x2_test_array[0:,:])

for i in range(len(x1_train_df)):
    for u in range(len(x_train['word 1'][i])):
        x1_train_df.iloc[i,u] = x_train['word 1'][i][u]

for i in range(len(x1_test_df)):
    for u in range(len(x_test['word 1'][i])):
        x1_test_df.iloc[i,u] = x_test['word 1'][i][u]
        
for i in range(len(x2_train_df)):
    for u in range(len(x_train['word 2'][i])):
        x2_train_df.iloc[i,u] = x_train['word 2'][i][u]
        
for i in range(len(x2_test_df)):
    for u in range(len(x_test['word 2'][i])):
        x2_test_df.iloc[i,u] = x_test['word 2'][i][u]
        

In [ ]:
#unzipping the vectors for the whole x_train and x_test datasets
#x_train_array = np.empty((len(x_train),100))
#x_train_df = pd.DataFrame(data=x_train_array[0:,:])

#x_test_array = np.empty((len(x_test),100))
#x_test_df = pd.DataFrame(data=x_test_array[0:,:])

#for i in range(len(x_train_df)):
 #   for u in range(len(x_train['word 2'][i])):
  #      x_train_df.iloc[i,u] = x_train['word 1'][i][u]
        
#for i in range(len(x_train_df)):
 #   for u in range(len(x_train['word 2'][i])):
  #      x_train_df.iloc[i,(50+u)] = x_train['word 2'][i][u]
              
#for i in range(len(x_test_df)):
 #   for u in range(len(x_test['word 1'][i])):
  #      x_test_df.iloc[i,u] = x_test['word 1'][i][u]
                       
#for i in range(len(x_test_df)):
 #   for u in range(len(x_test['word 2'][i])):
  #      x_test_df.iloc[i,(50+u)] = x_test['word 2'][i][u]

In [ ]:
#x1_train_tensor = torch.tensor(x1_train_df.values.astype(np.float32))#convert pd.DataFrame -> np.ndarray -> torch.tensor
#x2_train_tensor = torch.tensor(x2_train_df.values.astype(np.float32))
#syn_train_tensor = torch.tensor(syn_train.values.astype(np.float32))

    
#create tensor with features and targets
#train_tensor = torch.utils.data.TensorDataset(torch.cat((x1_train_tensor, x2_train_tensor), dim=0), syn_train_tensor)
#create iterable dataset with batches
#training_data_set = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

#x1_test_tensor = torch.tensor(x1_test_df.values.astype(np.float32))
#x2_test_tensor = torch.tensor(x2_test_df.values.astype(np.float32))
#syn_test_tensor = torch.tensor(syn_test.values.astype(np.float32))
#nonsyn_test_tensor = torch.tensor(nonsyn_test)

#test_tensor = torch.utils.data.TensorDataset(x1_test_tensor, x2_test_tensor, syn_test_tensor)
#testing_data_set = torch.utils.data.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = True)

In [4]:
X = carbon_df[['word 1', 'word 2']] #Input features used to make predictions
Y = carbon_df[['relationship']] #Target feature to be predicted 
 
x_train, x_test, syn_train, syn_test = train_test_split(X,Y, test_size = 0.2, shuffle = True) #split dataset into separate testing and training datasets

x_train.reset_index(inplace = True)
x_test.reset_index(inplace = True)
syn_train.reset_index(inplace = True)
syn_test.reset_index(inplace = True)

x1_train = x_train['word 1']
x2_train = x_train['word 2']
x1_test = x_test['word 1']
x2_test = x_test['word 2']

#Will probably need to save and load the x_train, x_test, syn_train, syn_test
#and call that in the respective train and test datasets

In [10]:
carbon_df

,Carbon_SynAntList_Full_Refined,word 1,word 2,relationship,label
0,1.0,"[1.0246487856, -5.6508703232, -1.4263287783, -...","[0.3912109137, -2.6639938354, -0.4191870987000...",1.0,1
1,2.0,"[1.0246487856, -5.6508703232, -1.4263287783, -...","[0.6780726314000001, -0.077852197, 3.356479167...",1.0,1
2,3.0,"[1.0246487856, -5.6508703232, -1.4263287783, -...","[-0.40175724030000004, 0.6633739471, -1.507220...",0.0,0
3,4.0,"[1.0246487856, -5.6508703232, -1.4263287783, -...","[2.6374275684, -0.8799803257000001, 1.95807564...",0.0,0
4,7.0,"[1.0246487856, -5.6508703232, -1.4263287783, -...","[-1.5558528900000002, 2.8244459629, -3.4161539...",0.0,0
...,...,...,...,...,...
1595,1994.0,"[-0.2082266062, 1.197119236, 1.1562068462, -5....","[-1.9820196629, 0.9932372570000001, 1.02549302...",0.0,0
1596,1995.0,"[-0.2082266062, 1.197119236, 1.1562068462, -5....","[0.3834728599, 0.7061271071, -0.00781521570000...",0.0,0
1597,1996.0,"[-0.2082266062, 1.197119236, 1.1562068462, -5....","[-2.8429896832, 1.9741824865000002, -0.3001447...",0.0,0
1598,1997.0,"[-0.2082266062, 1.197119236, 1.1562068462, -5....","[0.26439201830000003, 0.44663661720000003, -0....",0.0,0


In [ ]:
#This cell is to inspect what is being done in the SYN_Train_Dataset cell

#train_samples = [[] for i in range(len(x_train))]
        
#x1_train_array = np.empty((len(x_train),50))
#x1_train_df = pd.DataFrame(data=x1_train_array[0:,:])
#x2_train_array = np.empty((len(x_train),50))
#x2_train_df = pd.DataFrame(data=x2_train_array[0:,:])
       
#for i in range(len(x1_train_df)):
 #   for u in range(len(x_train['word 1'][i])):
  #      x1_train_df.iloc[i,u] = x_train['word 1'][i][u]
   #     x2_train_df.iloc[i,u] = x_train['word 2'][i][u]

#for i in range(len(x1_train_df)):
 #   train_samples[i] = [x1_train_df.iloc[i,:].tolist(), x2_train_df.iloc[i,:].tolist()], syn_train['relationship'][i]

In [6]:
class SYN_Train_Dataset(Dataset):
    
    def __init__(self):
        
        #we should probably load x_train, syn_train here after saving it after datasplit
        self.train_samples = [[] for i in range(len(x_train))]
        
        x1_train_array = np.empty((len(x_train),50))
        x1_train_df = pd.DataFrame(data=x1_train_array[0:,:])
        x2_train_array = np.empty((len(x_train),50))
        x2_train_df = pd.DataFrame(data=x2_train_array[0:,:])
       
        for i in range(len(x1_train_df)):
            for u in range(len(x_train['word 1'][i])):
                x1_train_df.iloc[i,u] = x_train['word 1'][i][u]
                x2_train_df.iloc[i,u] = x_train['word 2'][i][u] #save what x1 and x2 train_df look like outside the dataset

        for i in range(len(x1_train_df)):
            self.train_samples[i] = [x1_train_df.iloc[i,:].tolist(), x2_train_df.iloc[i,:].tolist()], syn_train['relationship'][i]
            #self.train_samples[i].append(x2_train_df.iloc[i,:].tolist())
            #self.train_samples[i].append(syn_train['relationship'][i])
        
      
    def __len__(self):
        
        return len(self.train_samples)
    
    def __getitem__(self, index):
        
        return self.train_samples[index]

if __name__ == '__main__':
    dataset = SYN_Train_Dataset()


In [9]:
 print(dataset[0])

([[0.9973859787, 2.3988208771, -3.0376405716, 0.8158480525, -1.6229007244, 2.8811962605, -0.9127098918000001, -2.7043669224, 0.12181437760000001, -0.5133670568, -1.649153471, -2.9331746101, 4.188202858, 3.0615022182, -2.3529634476, 0.43747541310000004, 3.2393746376, -1.5008270741, 1.1484955549, -1.5229882002, 3.7909066677, -0.7344305515, 2.5107870102, 2.8808379173, 1.1797090769, 0.4120798409, 2.9954190254, -2.3725163937, 3.0012643337, 2.3138353825, 0.8126859069, -0.7852016091, 3.0251886845, 1.9373266697, 0.4772277176, 0.41635900740000004, -7.3228983879000005, 2.3539719582, -0.0728377998, -1.4630441666, -1.9475537539, 0.4102407992, 3.4997196198, 0.0801593214, 2.729136467, -1.2905237675, 3.8139734268, -3.6601963043, 3.981307745, 5.4369807243], [0.7669314742000001, 0.2842409611, -1.5693807602, -2.947496891, -0.5005902052, -0.9371783137, -0.7468199134, 3.4563672543, 2.7787907124, -1.6619492769000002, -0.1412225217, -1.5308130979999999, -2.0223829746, -0.7647606730000001, 2.1933870316, -2.2

In [11]:
class SYN_Test_Dataset(Dataset):
    
    def __init__(self):
        
        #we should probably load x_test, syn_test here after saving it after datasplit
        self.test_samples = [[] for i in range(len(x_test))]
        
        x1_test_array = np.empty((len(x_test),50))
        x1_test_df = pd.DataFrame(data=x1_test_array[0:,:])
        x2_test_array = np.empty((len(x_test),50))
        x2_test_df = pd.DataFrame(data=x2_test_array[0:,:])

        for i in range(len(x1_test_df)):
            for u in range(len(x_test['word 1'][i])):
                x1_test_df.iloc[i,u] = x_test['word 1'][i][u]
                x2_test_df.iloc[i,u] = x_test['word 2'][i][u]

        for i in range(len(x1_test_df)):
            self.test_samples[i] = [x1_test_df.iloc[i,:].tolist(), x2_test_df.iloc[i,:].tolist()], syn_test['relationship'][i]
            
      
    def __len__(self):
        
        return len(self.test_samples)
    
    def __getitem__(self, idx):
        
        return self.test_samples[idx]

#if __name__ == '__main__':
 #   dataset = SYN_Test_Dataset()


In [12]:
#create tensor with features and targets
#create iterable dataset with batches
training_data_set = torch.utils.data.DataLoader(dataset = SYN_Train_Dataset(), batch_size = batch_size, shuffle = True)
testing_data_set = torch.utils.data.DataLoader(dataset = SYN_Test_Dataset(), batch_size = batch_size, shuffle = True)

In [ ]:
#Our NN will start with two branches, one that will take in x1_train, and the other to take in x2_train. 
#After embedding the input vectors for a few layers in their respective branches, the branches converge to a single branch, 
#which is when you concatenate x1_out and x2_out into syn_out. after going through the rest of the network, 
#loss will be calculated by comparing syn_out with syn_train (or syn_test)

In [ ]:
x1_train_tensor = torch.tensor(x1_train_df.values.astype(np.float32))
x2_train_tensor = torch.tensor(x2_train_df.values.astype(np.float32))
syn_train_tensor = torch.tensor(syn_train.values.astype(np.float32))
x1_test_tensor = torch.tensor(x1_test_df.values.astype(np.float32))
x2_test_tensor = torch.tensor(x2_test_df.values.astype(np.float32))
syn_test_tensor = torch.tensor(syn_test.values.astype(np.float32))

In [9]:
class SYN_TEST(nn.Module):

    def __init__(self, in_dims, out_dims):
        super(SYN_TEST, self).__init__()    
        
        self.x1_branch = nn.Sequential(
        nn.Linear(in_dims, 50)
        nn.Dropout(0.2), #to limit overfitting
        nn.Linear(50,100), #expand
        nn.Linear(100,300),
        nn.Linear(300,100),
        nn.Linear(100,50)) #compress
        
        #nn.Softplus()
        
        self.x2_branch = nn.Sequential(
        nn.Linear(in_dims, 50)
        nn.Dropout(0.2), #to limit overfitting
        nn.Linear(50,100), #expand
        nn.Linear(100,300),
        nn.Linear(300,100),
        nn.Linear(100,50)) #compress
        
        #hidden layers
        self.hidden_layer = nn.Linear(100, 32)
        self.hidden_layer1 = nn.Linear(32, 16)
        
        #output layer
        self.output_layer = nn.Linear(16,2)
   
    def forward(self, x1_in, x2_in):
       
        #pass each embedded data through each branch
        x1_out = self.x1_branch(x1_in)
        x2_out = self.x2_branch(x2_in)
        
        #concatenate outputs
        combined_out = torch.cat((x1_out, x2_out), dim = 1) #dim = 0 is end to end cat and dim = 1 is side by side cat
        
        #pass through hidden layers
        out = self.hidden_layer(combined_out)
        out = self.hidden_layer1(out)
        
        #subspace layer would be here
        
        #pass to output layer
        syn_out = self.output_layer(out)
        
        return syn_out
    
        

In [17]:
def train_model(model, training_data_set, optimizer):
    train_losses = []
    syn_train_losses = []
    train_epoch_loss = []
    syn_train_epoch_loss = []
    
    syn_losses = []
    train_total = 0
    
    #switch model to training mode
    model.train()
    syn_criterion = nn.MSELoss()
    
    #for features, labels in training_data_set: 
    
    #may need to change above "features" portion here to accomodate for our custom dataset
    #below is proposed alternative
    
    for i, data in enumerate(training_data_set,0):
        
        features, labels = data
        
        #have been encountering an issue where the data is not a double() but a float()
        features, labels = np.double(features), np.double(labels)
        
        model.zero_grad() #zero out any gradients from prior loops 
        syn_out = model(features) #gather model predictions for this loop
        
        #calculate error in the predictions
        syn_loss = syn_criterion(syn_out, labels)
        total_loss = syn_loss
        
        #BACKPROPAGATE LIKE A MF
        torch.autograd.backward([syn_loss])
        optimizer.step()
        
        #save loss for this batch
        train_losses.append(total_loss.item())
        train_total+=1
        
        syn_train_losses.append(syn_loss.item())
        
    #calculate and save total error for this epoch of training
    epoch_loss = sum(train_losses)/train_total
    train_epoch_loss.append(epoch_loss)
    
    syn_train_epoch_loss.append(sum(syn_train_losses)/train_total)
    
    #update progress bar
    print(f"Total Epoch Training Loss is: {train_epoch_loss}")
    
    return train_epoch_loss, syn_train_epoch_loss


In [12]:
def eval_model(model, testing_data_set, optimizer):
    #evaluate the model
    model.eval()
    
    syn_criterion = nn.MSELoss()
    #accuracy = #total number of correct predictions divided by the total number of predictions

    #don't update nodes during evaluation b/c not training
    with torch.no_grad():
        test_losses = []
        syn_test_losses = []
        #syn_test_acc_list = []
        
        test_total = 0

        #for inputs, labels in testing_data_set:
        #similar change to the train_model portion due to the nature of our data
        #inputs = inputs.to(device)
        #labels = labels.to(device)
        
        for i, data in enumerate(testing_data_set,0):
        
            inputs, labels = data
        
            #have been encountering an issue where the data is not a double() but a float()
            inputs, labels = np.double(inputs), np.double(labels)
            
            syn_out = model(inputs)

            # calculate loss per batch of testing data
            syn_test_loss = syn_criterion(syn_out, labels)
            
            test_loss = syn_test_loss
            
            test_losses.append(test_loss.item())
            syn_test_losses.append(syn_test_loss.item())
            test_total += 1 
            #syn_acc = accuracy(syn_out)
            #syn_test_acc_list.append(syn_acc.item())

        test_epoch_loss = sum(test_losses)/test_total
        syn_test_epoch_loss = sum(syn_test_losses)/test_total
        
        #syn_epoch_acc = sum(syn_test_acc_list)/test_total

        print(f"Total Epoch Testing Loss is: {test_epoch_loss}")
        #print(f"Epoch MAPE: Syn = {syn_epoch_acc}")
    
    return test_epoch_loss, syn_test_epoch_loss, #syn_epoch_acc




In [13]:
# Instantiate our beautiful NN model
# takes in 
# predicts synonymy
model = SYN_TEST(in_dims = 50, out_dims = 2).to(device)

#define the optimizer
optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)



In [18]:
#empty list to hold loss per epoch
train_epoch_losses = []
syn_train_epoch_losses = []

test_epoch_losses = []
syn_test_epoch_losses = []

syn_test_epoch_accuracies = []


for epoch in range(num_epochs):
    
    train_epoch_loss, syn_train_epoch_loss  = train_model(model = model, training_data_set = training_data_set, optimizer = optimizer)
    
    train_epoch_losses.append(train_epoch_loss)
    syn_train_epoch_losses.append(syn_train_epoch_loss)
   
    test_epoch_loss, syn_test_epoch_loss = eval_model(model = model, testing_data_set = testing_data_set, optimizer = optimizer)
    #syn_epoch_acc
    
    test_epoch_losses.append(test_epoch_loss)
    syn_test_epoch_losses.append(syn_test_epoch_loss)
    
    #pce_test_epoch_accuracies.append(pce_epoch_acc)
    #voc_test_epoch_accuracies.append(voc_epoch_acc)
    #jsc_test_epoch_accuracies.append(jsc_epoch_acc)
    #ff_test_epoch_accuracies.append(ff_epoch_acc)
    

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])


ValueError: only one element tensors can be converted to Python scalars

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.title("Total Training & Testing Error")
ax.set_xlabel('Epoch')
ax.set_ylabel('Total MSE Loss')
plt.show()

#fig, ax = plt.subplots(figsize = (8,6))
# plt.plot(epochs, train_epoch_accuracy, c = 'k', label = 'training accuracy')
#plt.plot(epochs, pce_test_epoch_accuracies, c = 'k', label = 'pce accuracy')
#plt.plot(epochs, voc_test_epoch_accuracies, c = 'r', label = 'voc accuracy')
#plt.plot(epochs, jsc_test_epoch_accuracies, c = 'g', label = 'jsc accuracy')
#plt.plot(epochs, ff_test_epoch_accuracies, c = 'b', label = 'ff accuracy')
#plt.legend(loc = 'lower right')
#plt.title("Branch Mean Absolute Percent Error")
#ax.set_xlabel('Epoch')
#ax.set_ylabel('MAPE')
#plt.show()

In [ ]:
xlin = ylin = np.arange(0, 100, 1)

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(PCE_out, pce_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 3)
ax.set_ylim(0, 3)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(Voc_out, voc_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 0.9)
ax.set_ylim(0, 0.9)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(Jsc_out, jsc_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()

fig, ax = plt.subplots(figsize = (8,6))

plt.scatter(FF_out, ff_labels)
plt.plot(xlin, ylin, c = 'k')
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
ax.set_xlabel("Predictions")
ax.set_ylabel("Ground Truth")
plt.show()
